# Data Science Topics

How to address issues that complicate data modeling and analytics.

### Table of Contents
1. [Missing Data](#missingdata)
1. [Unbalanced Data](#unbalanceddata)
1. [Curse of Dimensionality](#curseofdimensionality)
1. [Multicollinearity](#multicollinearity)

<a id="qs"></a>

___
# Questions to ask
<br>
References:<br>
[Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/)
___


### When presented with a dataset, what are the relevant questions?

Consider a marketing plan where money is spent on 3 independent advertising campaigns on TV, radio, and newspapers, and the resulting sales for the product is tracked for each campaign. So we have sales as a function of TV advertising, sales as a function of radio advertising, and sales as a function of newspaper advertising.

Important business questions are likely to be:
1. **Is there a relationship between advertising sales and budget?**
1. **How strong is the relationship?**
1. **Which media campaign contributes to sales?**
1. **How large is the effect of each medium on sales?**
1. **How accurately can we predict future sales?**
1. **Is the relationship linear?**
1. **Is there synergy among the advertising media?**

<a id="missingdata"></a>

___
# Missing Data
<br>
References: <br>
[https://www.utexas.edu/cola/prc/_files/cs/Missing-Data.pdf](https://www.utexas.edu/cola/prc/_files/cs/Missing-Data.pdf)<br>
[https://github.com/hammerlab/fancyimpute](https://github.com/hammerlab/fancyimpute)<br>
[Schafer & Graham 2002](http://www.nyu.edu/classes/shrout/G89-2247/Schafer&Graham2002.pdf)<br>
[http://cmhsr.wustl.edu/Resources/Documents/Imputing...pdf](http://cmhsr.wustl.edu/Resources/Documents/Imputing%20missing%20data_A%20comparison%20of%20methods%20for%20Social%20Work%20Researchers.pdf)
___

## Overview of procedure
1. Identify relationships in missing data
 * ***Missing completely at random (MCAR):*** missingness does not depend on missing variable or other variables
 * ***Missing at random (MAR):*** missingness depends on other variables but not missing variable
 * ***Missing not at random (MNAR):*** missingness depends on missing variable
<br><br>
1. Select appropriate method to impute or delete missing values

## 1. Identify relationships

First, check relationships between missing data variable and other variables using only examples/rows with non-missing data: 
- If there is a strong correlation between missing variable and other independent variables then it is unlikely to add any additional predictive power and can be discarded.
- If there is a strong correlation between missing variable and dependent variable (output feature) then it should definitely be kept and a suitable method for imputation found.
- If there are no strong correlations then the feature should be kept and dealt with appropriately as it may still have some predictive ability. 

Create dummy boolean/Bernoulli variable to describe missingness:
- If missingness correlates strongly with other independent variables then data can be interpreted as MAR.
- If missingness correlates strongly with output variable, then this feature may be replaced with the missingness variable. For classification, compare distribution of output values between missing and non-missing examples, if significantly different, then the missingness likely has some predictive power.
- To determine MNAR would require some range of potential values to be known.

For data MNAR:
- Often not possible to know if MNAR is true, unless there is a known range of values that are missing.
- Sometimes it is possible to predict missingness using other variables. For example, if high income earners don't report income, they may be identified by education or investments. 
- It may not be necessary to distinguish MNAR as some imputation methods for MAR also work here. 

For data MCAR or MAR:
- Some of the methods for imputation/deletion will work in either case.

## 2. Methods for addressing missing values

### Not recommended
- ***Single imputation with mean or mode:*** reduces variability, weakens correlation and covariance because it ignores relationships between variables including the output variable.

### If data are missing completely at random (MCAR)
- ***Listwise deletion:*** remove examples (rows) with missing data. Disadvantages include reduced statistical power if data are MCAR, if not MCAR this may introduce bias as the subsample data is not a good representation of the original data.  List-wise deletion can produce unbiased linear regression parameters as long as missingness is not a function of outcome variable. If dataset is small, then statistical power will be lost.

### If data are missing at random (MAR)
- ***Regression imputation:*** replacing missing variables with regression using other indpendent variables (perhaps those that have a high correlation with a dummy variable expressing missingness). Repeat regression until values between iterations converge. Overestimates model fit and correlation and reduces variability which can hurt predictive power.
<br><br>
- ***Single imputation with a conditional distribution:*** Use a highly correlated variable $(X)$ to construct a distribution of values for the missing variable $(Y)$ then draw values of $Y$ from this distribution (*conditional probability*),
<br><br>
$$P(Y_{mis} | Y_{obs};\theta) = \frac{P(Y_{obs};\theta | Y_{mis})}{P(Y_{obs};\theta)}$$
<br>
Computing from a conditional distribution is essentially unbiased under MCAR or MAR but potentially biased under MNAR. This method has the drawback of understating uncertainty as the fact that missing values are guesses is not included in the analysis. Although using a conditional distribution for imputation does a better job of preserving uncertainty. <br><br>Single imputation is reasonable if a dataset with many features is missing a small percentage of values that are spread evenly across the dataset.
<br><br>
- ***Multiple imputation with chained equations (MICE):*** This method involves filling in the missing values multiple times to create several complete datasets. Creating several estimates for missing values better characterizes the error in imputation. <br><br>In MICE, a series of regression models are run whereby each missing value is modeled conditional upon the other variables. So each variable is modeled according to its distribution, binary variables are modeled with logistic regression and continuous with linear regression.<br><br>Here is the process:
 1. A simple imputation (mean) is performed for all missing values in the dataset.
 1. One of the variables with missing data $(Y)$ is kept missing.
 1. The observed values for $Y$ are regressed with the other variables (all or some) in the imputed data.
 1. The missing values in $Y$ are then imputed with predictions from this regression.
 1. The previous steps are repeated for each variable with missing data.
 1. Again, this process is repeated with the just-imputed values until convergence (typically 10 cycles are used).
 1. After obtaining one "complete" dataset, repeat the entire procedure with missing variables imputed in a different order to obtain additional "complete" datasets, typically 5-10.
 1. The final analysis is performed on each of these datasets to provide an estimate of the error that arises from the imputation process.
 <br><br>
- ***Soft imputation:*** For datasets with high dimensionality and very few observations. Assumes the parameter describing the missing values $(Z)$ lies in a a much lower dimensional manifold. The problem becomes minimizing the nuclear norm $||Z||_{*}$ subject to the following optimization, problem,
$$ \sum_{(i,j) \in \Omega} (X_{ij} - X_{ij})^{2} \le \delta $$
 Which can be expressed in Lagrange form,
$$ \min_{Z} \sum_{(i,j) \in \Omega} (X_{ij} - X_{ij})^{2} + \lambda \ ||Z||_{*} $$
 Soft impute is an algorithm for nuclear norm regularization. It repeatedly replaces missing entries with the current guess, then updates them by solving the Lagrange equation above. This equation is solved with SVD of the truncated dataset that was used to construct $Z$.
<br><br>
- ***k-nearest neighbors:*** This method is appropriate for discrete and continuous variables. One drawback is the need to search through large datasets (many examples) to find the closest values, this can be mitigated by using a small training set or other techniques. Not well-suited to datasets with high dimensionality.

### If data are missing not at random (MNAR)
- Try imputation methods appropriate for MAR and see if model behaves as expected.
- Otherwise, collect new data.

## Appendix

### Maximum likelihood for missing data
Rather than imputing missing values, maximum likelihood computes parameters that maximize the log likelihood of obtaining the data set. The idea is to select the parameters $\theta$ that maximize,
$$ L = \prod_{i=1}^{n} f_{i} (y_{i1}, y_{i2},..., y_{ik}; \theta) $$
where $f$ is the joint probability function for observation $i$ and $\theta$ is the set of parameters to be estimated.

For example, if there are $m$ observations with complete data and $m-n$ are missing data for the first two variables (1 and 2), the likelihood to be maximized is,
$$ L = \prod_{i=1}^{m} f_{i} (y_{i1}, y_{i2},..., y_{ik}; \theta) \prod_{i=m+1}^{n} f_{i} (y_{i3}, y_{i4},..., y_{ik}; \theta) $$

Once these parameters are determined, a linear model can be constructed such as ordinary least squares.

### Summary of single imputation methods
<img src="singleimp.png">

### *Python library with additional methods*

# fancyimpute

A variety of matrix completion and imputation algorithms implemented in Python.

## Usage

```python
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute

# X is the complete data matrix
# X_incomplete has the same values as X except a subset have been replace with NaN

# Use 3 nearest rows which have a feature to fill in each row's missing features
X_filled_knn = KNN(k=3).complete(X_incomplete)

# matrix completion using convex optimization to find low-rank solution
# that still matches observed values. Slow!
X_filled_nnm = NuclearNormMinimization().complete(X_incomplete)

# Instead of solving the nuclear norm objective directly, instead
# induce sparsity using singular value thresholding
X_filled_softimpute = SoftImpute().complete(X_incomplete_normalized)

# print mean squared error for the three imputation methods above
nnm_mse = ((X_filled_nnm[missing_mask] - X[missing_mask]) ** 2).mean()
print("Nuclear norm minimization MSE: %f" % nnm_mse)

softImpute_mse = ((X_filled_softimpute[missing_mask] - X[missing_mask]) ** 2).mean()
print("SoftImpute MSE: %f" % softImpute_mse)

knn_mse = ((X_filled_knn[missing_mask] - X[missing_mask]) ** 2).mean()
print("knnImpute MSE: %f" % knn_mse)
```

## Algorithms

* `SimpleFill`: Replaces missing entries with the mean or median of each column.

* `KNN`: Nearest neighbor imputations which weights samples using the mean squared difference
on features for which two rows both have observed data.

* `SoftImpute`: Matrix completion by iterative soft thresholding of SVD decompositions. Inspired by the [softImpute](https://web.stanford.edu/~hastie/swData/softImpute/vignette.html) package for R, which is based on [Spectral Regularization Algorithms for Learning Large Incomplete Matrices](http://web.stanford.edu/~hastie/Papers/mazumder10a.pdf) by Mazumder et. al.

* `IterativeSVD`: Matrix completion by iterative low-rank SVD decomposition. Should be similar to SVDimpute from [Missing value estimation methods for DNA microarrays](http://www.ncbi.nlm.nih.gov/pubmed/11395428) by Troyanskaya et. al.

* `MICE`: Reimplementation of [Multiple Imputation by Chained Equations](http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3074241/).

* `MatrixFactorization`: Direct factorization of the incomplete matrix into low-rank `U` and `V`, with an L1 sparsity penalty on the elements of `U` and an L2 penalty on the elements of `V`. Solved by gradient descent.

* `NuclearNormMinimization`: Simple implementation of [Exact Matrix Completion via Convex Optimization](http://statweb.stanford.edu/~candes/papers/MatrixCompletion.pdf
) by Emmanuel Candes and Benjamin Recht using [cvxpy](http://www.cvxpy.org/en/latest/). Too slow for large matrices.

* `BiScaler`: Iterative estimation of row/column means and standard deviations to get doubly normalized
matrix. Not guaranteed to converge but works well in practice. Taken from [Matrix Completion and Low-Rank SVD via Fast Alternating Least Squares](http://arxiv.org/abs/1410.2596).

<a id="unbalanceddata"></a>

___
# Unbalanced Data
<br>
References:<br>
[https://github.com/scikit-learn-contrib/imbalanced-learn](https://github.com/scikit-learn-contrib/imbalanced-learn)<br>
[https://www.jair.org/media/953/live-953-2037-jair.pdf](https://www.jair.org/media/953/live-953-2037-jair.pdf)
___

Most classification algorithms will only perform optimally when the number of samples of each class is roughly the same. Highly skewed datasets, where the minority heavily outnumbered by one or more classes, have proven to be a challenge while at the same time becoming more and more common.

One way of addresing this issue is by resampling the dataset as to offset this imbalance with the hope of arriving and a more robust and fair decision boundary than you would otherwise.

Resampling techniques are divided in three categories: 
1. **Under-sampling** the majority class(es)
1. **Over-sampling** the minority class
1. **Ensemble sampling**.

Below is a list of the methods currently implemented in the `imlearn` module.
- Under-sampling
 1. Random majority under-sampling with replacement
 1. Extraction of majority-minority Tomek links
 1. Under-sampling with Cluster Centroids
 1. NearMiss-(1 & 2 & 3)
 1. Condensend Nearest Neighbour
 1. One-Sided Selection
 1. Neighboorhood Cleaning Rule
- Over-sampling
 1. Random minority over-sampling with replacement
 1. SMOTE - Synthetic Minority Over-sampling Technique
 1. bSMOTE(1&2) - Borderline SMOTE of types 1 and 2
 1. SVM_SMOTE - Support Vectors SMOTE
- Over-sampling followed by under-sampling
 1. SMOTE + Tomek links
 1. SMOTE + ENN
- Ensemble sampling
 1. EasyEnsemble
 1. BalanceCascade

## Algorithms

### Over-sampling

#### Synthetic Minority Oversampling Technique (SMOTE)

Creates synthetic samples from the minor class by selecting two or more similar instances (using a distance measure) and perturbing each instance one feature at a time by a random amount that is less than the difference between the neighboring instances.

In essence, the synthetic samples are generated along line segments connecting near nearest neighbors. This is done by finding the difference between the vectors representing nearest neighbors then multiplying by a constant between 0 and 1 and adding this amount to the example under consideration.

### Under-sampling

#### Condensed Nearest Neighbors (CNN)

Selects the subset of examples that are able to correctly classify the original dataset using a one-nearest neighbor rule. Given a training set $X$:
1. Scan $X$ to find an example $x$ whose nearest example in the undersampled dataset $U$ has a different label from $x$.
1. Remove $x$ from $X$ and add to $U$
1. Repeat scanning and removing until no more examples are added to $U$

<a id="curseofdimensionality"></a>

___

# Curse of dimensionality

References:<br>
[https://en.wikipedia.org/wiki/Curse_of_dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality)<br>
[http://cleverowl.uk/2016/02/06/curse-of-dimensionality-explained/](http://cleverowl.uk/2016/02/06/curse-of-dimensionality-explained/)<br>
[http://scikit-learn.org/stable/tutorial/statistical_inference/supervised_learning.html#the-curse-of-dimensionality](http://scikit-learn.org/stable/tutorial/statistical_inference/supervised_learning.html#the-curse-of-dimensionality)

___

Increasing the dimensionality of the available data can make it sparse. This sparsity is problematic for any method that requires statistical significance. In order to obtain a statistically sound and reliable result, the amount of data needed to support the result often grows exponentially with the dimensionality. 

Also organizing and searching data often relies on detecting areas where objects form groups with similar properties; in high dimensional data however all objects appear to be sparse and dissimilar in many ways which prevents common data organization strategies from being efficient. One example is the nearest neighbors method in machine learning.

<a id="multicollinearity"></a>

___
# Multicollinearity

References:<br>
[Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/)
___

<br>

## Collinearity

***Collinearity*** is the situation in which two or more predictor variables are closely related to one another. With high collinearity, a scatter plot of two predictor variables (or features) will resemble a line while low collinearity will show no discernable pattern.

For example, when trying to predict the `balance` on a credit card, there may a high degree of collinearity between credit `rating` and credit `limit` as predictors. As a result, there is a larger range of linear regression coefficent values for these two features that yield the minimum sum of squared residuals (RSS), credit `balance`. *In the figure below, notice the difference in scale for `limit` between the left and right, from [0.16,0.19] to [-0.1,0.2].*

<table border="1">
<tr><td>
<img src="collin.png" width="600">
</td></tr>
</table>

Collinearity reduces the accuracy of the estimates of the regression coefficients/parameters and causes the standard error for each $\hat{\beta}_j$ to grow, this also reduces the t-statistic and power of the hypothesis test, thus increasing the chance for type II error (to incorrectly fail to reject the null hypothesis that $\beta_j=0$ when the coefficient is actually non-zero).

**Detecting collinearity** can be done by examining the correlation matrix of the predictors, an extreme value (near -1 or +1) indicates that two of the variables are correlated and have collinearity.

## Multicollinearity

It is possible for three or more variables to have collinearity even if there is not a high correlation between any pair of them, this is ***multicollinearity***.

**Detecting multicollinearity** can be done with several methods:
1. ***Variance inflation factor*** - after fitting a multiple-variable least square regression to each of the $i$ predictors as a function of the other predictors, calculate VIF for each of these predictors from the coefficient of determination of the regression ($VIF_i = 1 / (1-R_i^2)$). The rule of thumb is if $VIF(\hat{\beta}_i) > 10$ then multicollinearity is high. The smallest possible value is 1, indicating absence of multicollinearity. This value can determine the feature that suffers from multicollinearity, then by iteratively removing the other features and calculating the correlation, the offending features can be identified.
<br><br>
1. ***Condition number test*** - the condition index measures the *ill-conditioning* of a matrix, meaning the inversion of the matrix is numerically unstable and highly sensitive to changes in the original matrix. It is calculated from the maximum eigenvalue divided by the minimum eigenvalue. If the condition number > 30, then multicollinearity exists. This method can also be used to identify which variables are causing the multicollinearity, see this [stack overflow post](http://stackoverflow.com/questions/25676145/capturing-high-multi-collinearity-in-statsmodels).

Multicollinearity does not necessarily hurt the accuracy of the prediction, however it will make it difficult to understand how individual predictors are influencing the prediction.

## Addressing collinearity

The two simplest solutions are:
1. Drop one of the problematic variables. Since there is duplicate information, this will likely not affect the predictive model. For example, dropping a variable from a linear model may not significantly effect the coefficient of determination $R^2$.
1. It may also be possible to combine the collinear variables into one by taking the mean average, for example.